In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.12.0


In [2]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)

x = np.random.rand(BATCH_SIZE, SEQ_LEN, EMB_SIZE).astype(np.float32)
print(x.shape)

(2, 100, 16)


# Создание простого LSTM слоя с помощью Keras

In [3]:
H_SIZE = 32

lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, recurrent_activation='sigmoid')

Metal device set to: Apple M2


In [4]:
y = lstm(x)
print(y.shape)

(2, 100, 32)


# Флаг return_sequences

In [5]:
lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, recurrent_activation='sigmoid')

In [6]:
y = lstm(x)
print(y.shape)

(2, 32)


# Реализация LSTM ячейки

In [7]:
class LSTMCell(tf.keras.Model):
    def __init__(self, h_size):
        super().__init__()
        self.h_size = h_size
        
        # input gate
        self.fcXI = tf.keras.layers.Dense(self.h_size)
        self.fcHI = tf.keras.layers.Dense(self.h_size, use_bias=False) # bias не нужен, так как он есть в fcXI
        
        # forget gate
        self.fcXF = tf.keras.layers.Dense(self.h_size)
        self.fcHF = tf.keras.layers.Dense(self.h_size, use_bias=False) # bias не нужен, так как он есть в fcXF
        
        # создание нового кандидата CH
        self.fcXC = tf.keras.layers.Dense(self.h_size)
        self.fcHC = tf.keras.layers.Dense(self.h_size, use_bias=False) # bias не нужен, так как он есть в fcXC
        
        # output gate
        self.fcXO = tf.keras.layers.Dense(self.h_size)
        self.fcHO = tf.keras.layers.Dense(self.h_size, use_bias=False) # bias не нужен, так как он есть в fcXO
        
    def call(self, x, h, c):
        i = tf.nn.sigmoid(self.fcXI(x) + self.fcHI(h))
        f = tf.nn.sigmoid(self.fcXF(x) + self.fcHF(h))
        o = tf.nn.sigmoid(self.fcXO(x) + self.fcHO(h))
        ch = tf.nn.tanh(self.fcXC(x) + self.fcHC(h))
        c = f * c + i * ch # поэлементное произведение и сумма
        h = o * tf.nn.tanh(c) # поэлементное произведение
        return h, c

In [9]:
# Рекурентная нейросеть
class LSTM(tf.keras.Model):
    def __init__(self, h_size, return_sequences=True):
        super().__init__()
        self.h_size = h_size
        self.return_sequences = return_sequences
        self.lstm_cell = LSTMCell(h_size)
        
    def call(self, x_all):
        batch, length, emb_size = x.shape
        h = tf.zeros((batch, self.h_size))
        c = tf.zeros((batch, self.h_size))
        
        if self.return_sequences:
            h_all = []

        
        for i in range(length):
            h, c = self.lstm_cell(x_all[:, i,  :], h, c)
            if self.return_sequences:
                h_all.append(h)
            
        if self.return_sequences:
            h_all = tf.transpose(tf.stack(h_all), [1, 0, 2])
            return h_all
        else:
            return h

    
    
lstm_my = LSTM(H_SIZE, False)

In [10]:
y = lstm_my(x)
print(y.shape)

(2, 32)


In [11]:
# Перед тем, как что-то присваивать в параметры модели, нужно чтобы они создались.
# Для этого можно вызвать либо инференс с каким-то входом, либо model.build(...)

lstm_my.lstm_cell.fcXI.kernel = lstm.weights[0][:, 0:32]
lstm_my.lstm_cell.fcXF.kernel = lstm.weights[0][:, 32:64]
lstm_my.lstm_cell.fcXC.kernel = lstm.weights[0][:, 64:96]
lstm_my.lstm_cell.fcXO.kernel = lstm.weights[0][:, 96:128]

lstm_my.lstm_cell.fcHI.kernel = lstm.weights[1][:, 0:32]
lstm_my.lstm_cell.fcHF.kernel = lstm.weights[1][:, 32:64]
lstm_my.lstm_cell.fcHC.kernel = lstm.weights[1][:, 64:96]
lstm_my.lstm_cell.fcHO.kernel = lstm.weights[1][:, 96:128]

lstm_my.lstm_cell.fcXI.bias = lstm.weights[2][0:32]
lstm_my.lstm_cell.fcXF.bias = lstm.weights[2][32:64]
lstm_my.lstm_cell.fcXC.bias = lstm.weights[2][64:96]
lstm_my.lstm_cell.fcXO.bias = lstm.weights[2][96:128]

y = lstm(x)
y_my = lstm_my(x)

print(np.max(np.abs(y.numpy() - y_my.numpy())))

7.4505806e-08
